In [1]:
import pandas as pd
import datetime as dt
import re
import numpy as np
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 100)

from matplotlib import pyplot as plt
%matplotlib inline 

import seaborn as sns
sns.set_style('darkgrid')

from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

import math

In [2]:
url = 'https://data.wprdc.org/dataset/d1eb0fcd-ba60-4407-9969-ceef464d0c00/resource/36c61dbc-9e23-4c77-9f09-ca544d3c6174/download/schedule_daily_detail.csv'
trip_detail = pd.read_csv(url)

In [3]:
def format_time(timestring):
    timestring = timestring.strip()
    end = re.findall('[A-Za-z]$', timestring)
    if len(end)==0:
        print(timestring + ' has no letters')
        try: 
            return dt.datetime.strptime(timestring, '%H%M')
        except:
            return pd.NaT
    elif end[0] in ['a', 'p']:
        timestring = timestring + 'm'
    elif end[0] == 'x':
        timestring = re.sub('x', 'am', timestring)
    else:
        print(timestring + ' has no letters')
        timestring = re.sub('[A-Za-z].*$', '', timestring)
    try: 
        return dt.datetime.strptime(timestring, '%I%M%p')
    except:
        return pd.NaT

In [4]:
starts = [format_time(t) for t in trip_detail['Trip.Start']]
trip_detail['start_dt'] = starts
ends = [format_time(t) for t in trip_detail['Trip.End']]
trip_detail['end_dt'] = ends
trip_detail['start_hour'] = trip_detail['start_dt'].dt.hour
trip_detail['end_hour'] = trip_detail['end_dt'].dt.hour

In [5]:
trip_detail.dtypes

Service.Context                    object
Schedule.Type                      object
Booking                             int64
Number.of.InSrv.Trip.s.             int64
Garage                              int64
Route                              object
Trip.Start                         object
Trip.End                           object
Trip.Block                         object
Trp.Off.Serv.Duration             float64
Trp.In.Serv.Duration              float64
Trp.Layover.Duration              float64
Trp.Total.Duration                float64
Trp.In.Serv.Dist.                 float64
Trp.Off.Serv.Dist.                float64
Trp.Dist.                         float64
Speed.Inbound                     float64
Speed.Outbound                    float64
Trip.Variant                       object
Trip.is.Pull.Out                  float64
Trip.is.Pull.In                   float64
Trip..                              int64
Day.Type                           object
start_dt                   datetim

In [6]:
def time_per(hour):
    peak_min1 = 7
    peak_max1 = 10
    peak_min2 = 16
    peak_max2 = 19
    if (peak_min1 <= hour < peak_max1):
        return "AM"
    elif (peak_min2 <= hour < peak_max2):
        return "PM"
    else:
        return "OFFPEAK"

def peak_or_not(hour, trip_variant):
    #print(trip_variant)
    peak_min1 = 7
    peak_max1 = 10
    peak_min2 = 16
    peak_max2 = 19
    try:
        trip_variant[0]
    except:
        return 0
    if (((peak_min1 <= hour < peak_max1) & (trip_variant[0]=="I"))
        |((peak_min2 <= hour < peak_max2)& (trip_variant[0]=="O"))):
        return 1
    else:
        return 0

trip_detail["TimePer"] = [time_per(h) for h in trip_detail['start_hour']]
trip_detail["Peak"] = [peak_or_not(trip_detail['start_hour'][i], trip_detail['Trip.Variant'][i]) for i in range(len(trip_detail))]

In [7]:
trip_detail.Route = [r.lstrip('0') for r in trip_detail.Route]

In [8]:
def get_peak_count(df):
    df = df[(df.Booking == 1909)&(trip_detail['Day.Type']=="Weekday")]
    df = df.groupby(['Route', 'Peak'])
    df = df.agg({"Trip..": "count", 'Trp.In.Serv.Duration': 'mean'}).reset_index()
    df.columns = ['Route', 'Peak', "TripCount", 'Trip_HRS_Service']
    return df

In [9]:
peak_counts = get_peak_count(trip_detail)
peak_counts.to_csv('../Outputs/trip_counts_peak.csv')

In [10]:
peak_counts.Route = [r.lstrip('0') for r in peak_counts.Route]
peak_counts

,Route,Peak,TripCount,Trip_HRS_Service
0,1,0,38,1.392474
1,1,1,10,1.489800
2,11,0,39,0.401538
3,11,1,11,0.377545
4,12,0,37,1.086405
...,...,...,...,...
191,Y46,1,13,1.139692
192,Y47,0,38,0.840447
193,Y47,1,10,0.860200
194,Y49,0,39,0.871410


In [11]:
int(False)

0

In [12]:
def get_from_peak_table(route, peak=True, trip_not_time=True):
    #print(route)
    try:
        p = peak_counts[((peak_counts['Route']==route)&(peak_counts['Peak']==int(peak)))]
    #   print(p)
    except:
        print(route + ' not found')
        return 0
    if trip_not_time:
        try:
            cnt = p["TripCount"].iloc[0]
        except:
    #        print(p + 'not indexed')
            return 0
    else:
        try:
            cnt = p["Trip_HRS_Service"].iloc[0]
        except:
    #        print(p + 'not indexed')
            return 0
    return cnt

# Modeling Ridership Estimate

In [13]:
path1 = "../Data/RouteMatrix.xlsx"
xls1 = pd.ExcelFile(path1)
sheet_list = xls1.sheet_names  # see all sheet names
m = pd.read_excel(xls1, 'Matrix',  header=0)

In [14]:
def to_float(num):
    try:
        float(num)
        return float(num)
    except ValueError:
        return np.nan

In [15]:
matrix = m.iloc[:,:-45]
conv_col = ['Weekday_Avg_Headway_Inbound_Peak', 'Weekday_Avg Ridership',
       'Weekday_Avg_Headway_Inbound_OffPeak',
       'Weekday_Avg_Headway_Outbound_Peak',
       'Weekday_Avg_Headway_Outbound_OffPeak', 'Weekday_Peak_No_Units',
       'Saturday_Avg Ridership', 'Saturday_Avg_Headway_Inbound',
       'Saturday_Avg_Headway_Outbound', 'Saturday_# of Units',
       'Sunday_Avg Ridership',
       'Sunday_Avg_Headway_Inbound', 'Sunday_Avg_Headway_Outbound',
       'Sunday_# of Units']
for c in conv_col:
    matrix[c] = [to_float(a) for a in matrix[c]]

In [16]:
matrix['Weekday_Trips'] = matrix[['Weekday_Trips_Outbound', 'Weekday_Trips_Inbound']].sum(axis=1)
matrix['Sat_Trips'] = matrix[['Saturday_Trips_Outbound','Saturday_Trips_Inbound']].sum(axis=1)
matrix['Sun_Trips'] = matrix[['Sunday_Trips_Outbound','Sunday_Trips_Inbound']].sum(axis=1)
matrix['Wkdy_Peak_Hdwy'] = matrix[['Weekday_Avg_Headway_Inbound_Peak','Weekday_Avg_Headway_Outbound_Peak']].dropna().min(axis=1)
matrix['Wkdy_Offpeak_Hdwy'] = matrix[['Weekday_Avg_Headway_Inbound_OffPeak','Weekday_Avg_Headway_Outbound_OffPeak']].dropna().max(axis=1)
matrix['Sat_Hdwy'] = matrix[['Saturday_Avg_Headway_Inbound','Saturday_Avg_Headway_Outbound']].dropna().mean(axis=1)
matrix['Sun_Hdwy'] = matrix[['Sunday_Avg_Headway_Inbound','Sunday_Avg_Headway_Outbound']].dropna().mean(axis=1)
matrix = matrix[['Route Number', 'Walkshed Population', 'Avg Equity Score', 'Public Transportation Commute', 
       'No Vehicle Households', 'Median Income', 'Total Ridership', 'Weekday_Avg Ridership',
                 'Saturday_Avg Ridership', 'Sunday_Avg Ridership', 'Weekday_Trips',
       'Sat_Trips',       'Sun_Trips',       'Wkdy_Peak_Hdwy',       'Wkdy_Offpeak_Hdwy',       'Sat_Hdwy', 'Sun_Hdwy']]
matrix['Peak_Trips'] = [get_from_peak_table(r, peak=True, trip_not_time=True) for r in matrix['Route Number']]
matrix['OffPeak_Trips'] = matrix['Weekday_Trips']-matrix['Peak_Trips']
matrix['Peak_Runtime'] = [get_from_peak_table(r, peak=True, trip_not_time=False) for r in matrix['Route Number']]
matrix['Offpeak_Runtime'] = [get_from_peak_table(r, peak=False, trip_not_time=False) for r in matrix['Route Number']]

In [17]:
reliable = pd.read_csv('https://data.wprdc.org/datastore/dump/00eb9600-69b5-4f11-b20a-8c8ddd8cfe7a')
rel_grp = reliable.groupby(['route', 'route_full_name', 'day_type'])
rel_grp = rel_grp.agg({"on_time_percent": "mean", 'month_start': 'count'}).reset_index()

In [18]:
def get_ontime(route, daytype):
    #print(route)
    try:
        p = rel_grp.loc[((rel_grp['route']==str(route))&(rel_grp['day_type']==daytype)),'on_time_percent'].iloc[0]
        return p*100
    except:
        print(route + ' not found')
        return 0

In [19]:
matrix['Wkdy_Ontime'] = [get_ontime(r, daytype='WEEKDAY') for r in matrix['Route Number']] 
matrix['Sat_Ontime'] = [get_ontime(r, daytype='SAT.') for r in matrix['Route Number']]
matrix['Sun_Ontime'] = [get_ontime(r, daytype='SUN.') for r in matrix['Route Number']]

In [20]:
matrix.head()

,Route Number,Walkshed Population,Avg Equity Score,Public Transportation Commute,No Vehicle Households,Median Income,Total Ridership,Weekday_Avg Ridership,Saturday_Avg Ridership,Sunday_Avg Ridership,Weekday_Trips,Sat_Trips,Sun_Trips,Wkdy_Peak_Hdwy,Wkdy_Offpeak_Hdwy,Sat_Hdwy,Sun_Hdwy,Peak_Trips,OffPeak_Trips,Peak_Runtime,Offpeak_Runtime,Wkdy_Ontime,Sat_Ontime,Sun_Ontime
0,52L,22917,0.2371,0.160638,0.193936,46782,110696,429.0,NaN,NaN,24,0,0,28.728311,NaN,NaN,NaN,8,16,0.920625,0.610500,67.119231,0.000000,0.000000
1,53,13208,0.2296,0.137600,0.222329,47659,17501,NaN,337.0,NaN,0,34,0,NaN,NaN,59.911111,NaN,0,0,0.000000,0.000000,0.000000,72.058205,8.922222
2,53L,37038,0.2072,0.146400,0.195652,53303,377123,1462.0,NaN,NaN,55,0,0,24.331790,59.248052,NaN,NaN,13,42,1.196077,1.028595,65.950256,0.000000,0.000000
3,55,20479,0.2350,0.079900,0.253619,34649,331946,1005.0,759.0,601.0,36,36,34,60.000000,60.061506,59.763636,60.244213,0,36,0.000000,0.000000,74.326410,72.450000,74.829487
4,56,32080,0.2380,0.108700,0.275747,35137,516895,1700.0,823.0,647.0,60,34,32,25.144712,49.667461,60.000000,60.127273,0,60,0.000000,0.000000,68.416154,73.678205,78.109744


In [21]:
matrix['Weekend_Avg_Ridership'] =  matrix[['Saturday_Avg Ridership','Sunday_Avg Ridership']].sum(axis=1)
matrix['Weekend_Trips'] =  matrix[['Sat_Trips','Sun_Trips']].sum(axis=1)
matrix['Weekend_Hdwy'] =  matrix[['Sat_Hdwy','Sun_Hdwy']].max(axis=1)
matrix['Weekend_Ontime'] =  matrix[['Sat_Ontime','Sun_Ontime']].mean(axis=1)

In [22]:
matrix_p7 = matrix[matrix['Route Number']=='P7']
matrix_p7['OffPeak_Trips'] = matrix_p7['OffPeak_Trips'] + 26
matrix_p7['Weekday_Trips'] = 55
matrix_p7['Wkdy_Offpeak_Hdwy'] = 30
matrix_p7['Weekend_Trips'] = 24
matrix_p7['Weekend_Hdwy'] = 70

In [23]:
matrix_p7

,Route Number,Walkshed Population,Avg Equity Score,Public Transportation Commute,No Vehicle Households,Median Income,Total Ridership,Weekday_Avg Ridership,Saturday_Avg Ridership,Sunday_Avg Ridership,Weekday_Trips,Sat_Trips,Sun_Trips,Wkdy_Peak_Hdwy,Wkdy_Offpeak_Hdwy,Sat_Hdwy,Sun_Hdwy,Peak_Trips,OffPeak_Trips,Peak_Runtime,Offpeak_Runtime,Wkdy_Ontime,Sat_Ontime,Sun_Ontime,Weekend_Avg_Ridership,Weekend_Trips,Weekend_Hdwy,Weekend_Ontime
34,P7,51470,0.2869,0.25,0.285881,40514,197203,764.0,NaN,NaN,55,0,0,24.344444,30,NaN,NaN,10,45,0.8699,0.843,57.640513,0.0,0.0,0.0,24,70,0.0


In [76]:
wkdy_matrix_p7 = matrix_p7.loc[matrix_p7['Weekday_Avg Ridership'].isna()==False,['Walkshed Population', 'Avg Equity Score',
       'Public Transportation Commute', 'No Vehicle Households',
       'Median Income', 'Wkdy_Ontime', 'Weekday_Trips','Wkdy_Peak_Hdwy',
       'Wkdy_Offpeak_Hdwy', 'Peak_Trips',
       'Peak_Runtime', 'Offpeak_Runtime']]

In [26]:
wkdy_x = matrix.loc[matrix['Weekday_Avg Ridership'].isna()==False,['Walkshed Population', 'Avg Equity Score',
       'Public Transportation Commute', 'No Vehicle Households',
       'Median Income', 'Wkdy_Ontime', 'Weekday_Trips','Wkdy_Peak_Hdwy',
       'Wkdy_Offpeak_Hdwy', 'Peak_Trips',
       'Peak_Runtime', 'Offpeak_Runtime']]
wkdy_y = matrix.loc[matrix['Weekday_Avg Ridership'].isna()==False,['Weekday_Avg Ridership']]
wkdy_x.fillna(wkdy_x.mean(), inplace=True)

In [27]:
wend_x = matrix.loc[(matrix['Weekend_Avg_Ridership'].isna()==False)&(matrix['Weekend_Avg_Ridership'] !=0),
                           ['Walkshed Population', 'Avg Equity Score',
       'Public Transportation Commute', 'No Vehicle Households',
       'Median Income', 'Weekend_Ontime', 'Weekend_Trips','Weekend_Hdwy',
       'Offpeak_Runtime']]
wend_y = matrix.loc[(matrix['Weekend_Avg_Ridership'].isna()==False)&(matrix['Weekend_Avg_Ridership'] !=0)
                    ,['Weekend_Avg_Ridership']]
wend_x.fillna(wend_x.mean(), inplace=True)

In [75]:
wend_matrix_p7 = matrix_p7.loc[:,
                           ['Walkshed Population', 'Avg Equity Score',
       'Public Transportation Commute', 'No Vehicle Households',
       'Median Income', 'Weekend_Ontime', 'Weekend_Trips','Weekend_Hdwy',
       'Offpeak_Runtime']]

# Modeling
## First a simple linear regression

In [29]:
X_train, X_test, y_train, y_test = train_test_split(wkdy_x, wkdy_y, test_size=0.30, random_state=40)

In [33]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred_train = lr_model.predict(X_train)
y_pred_test = lr_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = math.sqrt(mse)

print('R2 Train: {}'.format(r2_score(y_train, y_pred_train)))
print('R2 Test: {}'.format(r2_score(y_test, y_pred_test)))
print('RMSE: {}'.format(rmse))

R2 Train: 0.9740353130538549
R2 Test: -0.458977701278265
RMSE: 944.2811970741807


This model does horribly. Let's try with some regularization:
## Lasso

In [41]:
lasso_model = Lasso(random_state=1234,
                  tol=1)
lasso_model.fit(X_train, y_train)

y_pred_train = lasso_model.predict(X_train)
y_pred_test = lasso_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)
rmse = math.sqrt(mse)

print('R2 Train: {}'.format(r2_score(y_train, y_pred_train)))
print('R2 Test: {}'.format(r2_score(y_test, y_pred_test)))

print('RMSE: {}'.format(rmse))

R2 Train: 0.9648961061417406
R2 Test: 0.44677572134363386
RMSE: 581.470425029683


In [42]:
enet_mod = ElasticNet(alpha=1.0,
                  l1_ratio=0.5,
                  random_state=1234,
                  tol=1)
enet_mod.fit(X_train, y_train)

pred_train_= enet_mod.predict(X_train)
pred_test_= enet_mod.predict(X_test)

print('R2 Train: {}'.format(r2_score(y_train, pred_train_)))
print('R2 Test: {}'.format(r2_score(y_test, pred_test_)))
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test,pred_test_))))


R2 Train: 0.9504576081257767
R2 Test: 0.5910741212860098
RMSE: 499.9186197200794


Elastic net seems to be doing much better than the other regression models

In [43]:
enet = ElasticNet(alpha=1.0,
                  l1_ratio=0.5,
                  random_state=1234,
                  tol=1)
enet.fit(wkdy_x, wkdy_y)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=1234, selection='cyclic', tol=1, warm_start=False)

In [44]:
enet_mod.predict(wkdy_matrix_p7)

array([2255.56001828])

In [45]:
matrix

,Route Number,Walkshed Population,Avg Equity Score,Public Transportation Commute,No Vehicle Households,Median Income,Total Ridership,Weekday_Avg Ridership,Saturday_Avg Ridership,Sunday_Avg Ridership,Weekday_Trips,Sat_Trips,Sun_Trips,Wkdy_Peak_Hdwy,Wkdy_Offpeak_Hdwy,Sat_Hdwy,Sun_Hdwy,Peak_Trips,OffPeak_Trips,Peak_Runtime,Offpeak_Runtime,Wkdy_Ontime,Sat_Ontime,Sun_Ontime,Weekend_Avg_Ridership,Weekend_Trips,Weekend_Hdwy,Weekend_Ontime
0,52L,22917,0.2371,0.160638,0.193936,46782,110696,429.0,NaN,NaN,24,0,0,28.728311,NaN,NaN,NaN,8,16,0.920625,0.610500,67.119231,0.000000,0.000000,0.0,0,NaN,0.000000
1,53,13208,0.2296,0.137600,0.222329,47659,17501,NaN,337.0,NaN,0,34,0,NaN,NaN,59.911111,NaN,0,0,0.000000,0.000000,0.000000,72.058205,8.922222,337.0,34,59.911111,40.490214
2,53L,37038,0.2072,0.146400,0.195652,53303,377123,1462.0,NaN,NaN,55,0,0,24.331790,59.248052,NaN,NaN,13,42,1.196077,1.028595,65.950256,0.000000,0.000000,0.0,0,NaN,0.000000
3,55,20479,0.2350,0.079900,0.253619,34649,331946,1005.0,759.0,601.0,36,36,34,60.000000,60.061506,59.763636,60.244213,0,36,0.000000,0.000000,74.326410,72.450000,74.829487,1360.0,70,60.244213,73.639744
4,56,32080,0.2380,0.108700,0.275747,35137,516895,1700.0,823.0,647.0,60,34,32,25.144712,49.667461,60.000000,60.127273,0,60,0.000000,0.000000,68.416154,73.678205,78.109744,1470.0,66,60.127273,75.893974
5,57,21417,0.2628,0.173900,0.308560,33562,414276,1275.0,927.0,674.0,62,38,36,24.387500,47.589928,59.812500,60.410764,0,62,0.000000,0.000000,67.773333,63.192821,72.871026,1601.0,74,60.410764,68.031923
6,59,48108,0.2783,0.190900,0.290470,33826,724016,2142.0,1912.0,1308.0,38,36,28,60.000000,59.608302,60.406998,60.192449,0,38,0.000000,0.000000,67.063590,67.663846,67.563590,3220.0,64,60.406998,67.613718
7,60,14204,0.3020,0.129200,0.330309,25676,145956,566.0,NaN,NaN,30,0,0,59.750000,59.892857,NaN,NaN,0,30,0.000000,0.000000,80.990000,6.617778,0.000000,0.0,0,NaN,3.308889
8,61A,79046,0.2546,0.208800,0.247956,37280,1560788,4895.0,3210.0,2381.0,110,88,64,15.837414,23.463483,24.820209,36.667135,20,90,1.170150,1.020189,60.572051,68.983333,66.023846,5591.0,152,36.667135,67.503590
9,61B,71049,0.2418,0.199200,0.238222,38560,1380803,4394.0,2708.0,1935.0,114,98,66,16.572146,24.426284,24.908870,36.096045,20,94,0.940150,0.824468,60.850000,68.796154,68.567692,4643.0,164,36.096045,68.681923


In [46]:
matrix[matrix['Route Number']=='P7']

,Route Number,Walkshed Population,Avg Equity Score,Public Transportation Commute,No Vehicle Households,Median Income,Total Ridership,Weekday_Avg Ridership,Saturday_Avg Ridership,Sunday_Avg Ridership,Weekday_Trips,Sat_Trips,Sun_Trips,Wkdy_Peak_Hdwy,Wkdy_Offpeak_Hdwy,Sat_Hdwy,Sun_Hdwy,Peak_Trips,OffPeak_Trips,Peak_Runtime,Offpeak_Runtime,Wkdy_Ontime,Sat_Ontime,Sun_Ontime,Weekend_Avg_Ridership,Weekend_Trips,Weekend_Hdwy,Weekend_Ontime
34,P7,51470,0.2869,0.25,0.285881,40514,197203,764.0,NaN,NaN,29,0,0,24.344444,NaN,NaN,NaN,10,19,0.8699,0.843,57.640513,0.0,0.0,0.0,0,NaN,0.0


In [47]:
764/29#*55

26.344827586206897

In [51]:
wkdy_x

,Walkshed Population,Avg Equity Score,Public Transportation Commute,No Vehicle Households,Median Income,Wkdy_Ontime,Weekday_Trips,Wkdy_Peak_Hdwy,Wkdy_Offpeak_Hdwy,Peak_Trips,Peak_Runtime,Offpeak_Runtime
0,22917,0.2371,0.160638,0.193936,46782,67.119231,24,28.728311,40.787176,8,0.920625,0.610500
2,37038,0.2072,0.146400,0.195652,53303,65.950256,55,24.331790,59.248052,13,1.196077,1.028595
3,20479,0.2350,0.079900,0.253619,34649,74.326410,36,60.000000,60.061506,0,0.000000,0.000000
4,32080,0.2380,0.108700,0.275747,35137,68.416154,60,25.144712,49.667461,0,0.000000,0.000000
5,21417,0.2628,0.173900,0.308560,33562,67.773333,62,24.387500,47.589928,0,0.000000,0.000000
6,48108,0.2783,0.190900,0.290470,33826,67.063590,38,60.000000,59.608302,0,0.000000,0.000000
7,14204,0.3020,0.129200,0.330309,25676,80.990000,30,59.750000,59.892857,0,0.000000,0.000000
8,79046,0.2546,0.208800,0.247956,37280,60.572051,110,15.837414,23.463483,20,1.170150,1.020189
9,71049,0.2418,0.199200,0.238222,38560,60.850000,114,16.572146,24.426284,20,0.940150,0.824468
10,74631,0.2721,0.197600,0.243919,36634,58.549231,114,14.992239,23.850554,21,1.228571,1.070355


In [48]:
pred_enet = enet.predict(wkdy_x)
print(np.sqrt(mean_squared_error(wkdy_y,pred_enet)))
print(r2_score(wkdy_y, pred_enet))

516.5417193027964
0.9438685482040371


In [49]:
print( list(zip(matrix['Route Number'], pred_enet) ))

[('52L', 167.64341811323902), (53, 1333.6067491632334), ('53L', 775.9544987180755), (55, 1677.4428890426864), (56, 1661.0783495590535), (57, 1432.7891786047835), (59, 596.3234381704231), (60, 4795.270332258749), ('61A', 4770.331597845998), ('61B', 4955.045063301532), ('61C', 5572.755226177253), ('61D', 2191.5844104541407), (64, 287.277175145442), (65, 2292.2858240522746), (67, -584.4551403993851), (68, 2128.568703834294), (69, -545.2779107290362), (71, 5342.88923155932), ('71C', 5513.901681413161), ('71D', 1366.360517911703), (74, 2454.8493837208644), (77, 3037.152255880712), (79, 2350.330535409321), (86, 9783.00667793325), (93, 1142.5864626058114), ('P1', 646.8918139657635), ('P12', -27.20578154806526), ('P16', 952.3845536862602), ('P17', 4329.524020039335), ('P2', 277.2862838787464), ('P3', 1572.034241108665), ('P67', 390.62844675842916), ('P68', 1168.0949009393796), ('P69', 695.4373112904611), ('P7', 1004.2400892635544), ('P71', 1773.3835708705867)]


In [52]:
enet.predict(wkdy_matrix_p7)

array([2171.87360059])

In [53]:
print( list(zip(wkdy_x.columns, enet.coef_) ))

[('Walkshed Population', 0.014507276688218924), ('Avg Equity Score', -0.0), ('Public Transportation Commute', -0.0), ('No Vehicle Households', -0.0), ('Median Income', -0.03071522607827541), ('Wkdy_Ontime', -21.373093727210787), ('Weekday_Trips', 36.72489609147823), ('Wkdy_Peak_Hdwy', 8.798882947149757), ('Wkdy_Offpeak_Hdwy', -4.536071448201992), ('Peak_Trips', 19.809061401121692), ('Peak_Runtime', 26.55317512930801), ('Offpeak_Runtime', 33.505712708629865)]


In [54]:
offpeak_factor = 36.72489609147823-19.809061401121692
offpeak_factor

16.915834690356537

Weekend prediction

In [55]:
X_train_wnd, X_test_wnd, y_train_wnd, y_test_wnd = train_test_split(wend_x, wend_y, test_size=0.30, random_state=40)

In [56]:
enet_wend_train = ElasticNet(alpha=1.0,
                  l1_ratio=0.5,
                  random_state=1234,
                  tol=1)
enet_wend_train.fit(X_train_wnd, y_train_wnd)

pred_train_= enet_wend_train.predict(X_train_wnd)
pred_test_= enet_wend_train.predict(X_test_wnd)

print('R2 Train: {}'.format(r2_score(y_train_wnd, pred_train_)))
print('R2 Test: {}'.format(r2_score(y_test_wnd, pred_test_)))
print('RMSE: {}'.format(np.sqrt(mean_squared_error(y_test_wnd,pred_test_))))


R2 Train: 0.8533161966239351
R2 Test: 0.09691491692358434
RMSE: 2560.4935616641937


In [57]:
enet_wend = ElasticNet(alpha=1.0,
                  l1_ratio=0.5,
                  random_state=1234,
                  tol=1)
enet_wend.fit(wend_x, wend_y)
pred_enet_wend = enet_wend.predict(wend_x)
print(np.sqrt(mean_squared_error(wend_y,pred_enet_wend)))
print(r2_score(wend_y, pred_enet_wend))

856.4792805595032
0.8710684420903879


In [58]:
enet_wend.predict(wend_matrix_p7)

array([667.22544166])

In [59]:
print( list(zip(wend_x.columns, enet_wend.coef_) ))

[('Walkshed Population', 0.0036724296717589255), ('Avg Equity Score', 6.197624387666239), ('Public Transportation Commute', -0.9495614743161057), ('No Vehicle Households', -0.0), ('Median Income', -0.042345297568761556), ('Weekend_Ontime', -9.248721804976176), ('Weekend_Trips', 37.57546128609923), ('Weekend_Hdwy', -12.60352032726758), ('Offpeak_Runtime', 122.41728288173253)]


## Predicting New Braddock Service

Using a combination of P1 runtime and a conversion rate on driving time at rush hour, we have the following service characteristics (following P7 headway, adding trips to fill the peak hours, and off peak and weekend to fill 8-9 hours a day of service)

**Headway**
* Proposed Peak HW	20
* Proposed Wkdy Off Peak HW	30

**Frequency** 

*1W Wkday Trips*
* Peak trips	18
* Offpeak trips	18
* Total	36

*Weekend*
* Proposed HW	45
* Trips/day	12

**RT Time**	
* 1W Time	51
* Recovery*	10%
* Build RT Time	112

Using pieced together bits from the P1 and P68 walkshed analysis, we have the following:
* Romute Number	: New_P
* Walkshed Population	: 57731
* Avg Equity Score	: 0.27
* Public Transportation Commute	: 0.091579914
* No Vehicle Households	: 0.103791724
* Median Income	: 43196.6894


In [92]:
new_braddock_wkdy = wkdy_matrix_p7.copy()
new_braddock_wkdy['Walkshed Population'] = 57731
new_braddock_wkdy['Avg Equity Score'] = 0.27
new_braddock_wkdy['Public Transportation Commute'] = 0.091579914
new_braddock_wkdy['No Vehicle Households'] = 0.103791724
new_braddock_wkdy['Median Income'] = 43196.69

new_braddock_wkdy['Peak_Trips'] = 18
new_braddock_wkdy['Weekday_Trips'] = 36
new_braddock_wkdy['Wkdy_Peak_Hdwy'] = 20
new_braddock_wkdy['Wkdy_Offpeak_Hdwy'] = 30
new_braddock_wkdy['Peak_Runtime'] = 58/60.0
new_braddock_wkdy['Offpeak_Runtime'] = (58-(24-18))/60.0
new_braddock_wkdy

,Walkshed Population,Avg Equity Score,Public Transportation Commute,No Vehicle Households,Median Income,Wkdy_Ontime,Weekday_Trips,Wkdy_Peak_Hdwy,Wkdy_Offpeak_Hdwy,Peak_Trips,Peak_Runtime,Offpeak_Runtime
34,57731,0.27,0.09158,0.103792,43196.69,57.640513,36,20,30,18,0.966667,0.866667


In [95]:
enet_mod.predict(new_braddock_wkdy)

array([1615.89330451])

In [93]:
new_braddock_wend = wend_matrix_p7.copy()
new_braddock_wend['Walkshed Population'] = 57731
new_braddock_wend['Avg Equity Score'] = 0.27
new_braddock_wend['Public Transportation Commute'] = 0.091579914
new_braddock_wend['No Vehicle Households'] = 0.103791724
new_braddock_wend['Median Income'] = 43196.69
new_braddock_wend['Weekend_Trips'] = 12
new_braddock_wend['Weekend_Hdwy'] = 45
new_braddock_wkdy['Offpeak_Runtime'] = (58-(24-18))/60.0
new_braddock_wend

,Walkshed Population,Avg Equity Score,Public Transportation Commute,No Vehicle Households,Median Income,Weekend_Ontime,Weekend_Trips,Weekend_Hdwy,Offpeak_Runtime
34,57731,0.27,0.09158,0.103792,43196.69,0.0,12,45,0.843


In [94]:
enet_wend.predict(new_braddock_wend)

array([440.84738001])

In [96]:
1615*52*5+440*52*2

465660